# Fake News Exploratory Data Analysis

Get subset of FakeNewCorpus data set.

In [ ]:
import pandas as pd
import regex as re

# Import 250 articles as pandas df
raw_data = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv', dtype={"content": "string"}, index_col=0) 
raw_data.head()

### Cleaning and preprocessing

Perform basic cleaning: remove urls, dates, numbers, emails.

In [ ]:
data250 = raw_data.copy(deep = True) # Create a deep copy of dataframe, before modifying

In [ ]:
from functions import clean_dataframe
clean_dataframe(data250)
data250['content'].head()

Tokenize the text
Remove stopwords and compute reduction of vocabulary rate after having removed them
Stem words and compute reduction rate of vocabulary

In [ ]:
from functions import preprocess
preprocess(data250)
data250['content']

Next, we define functions that can visualize our dataset

### Visualization of data set

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from functions import pair_keyword_type, scatterplot_keyword_type
scatterplot_keyword_type("num", data250)

In [ ]:
from functions import boxplot_keyword_type
boxplot_keyword_type('trump', data250)

In [ ]:
from functions import barplot_keyword_type
barplot_keyword_type('num', data250) 

# FROM HERE DOWN CODE HASN*T BEEN STRUCTURED

### Get subset of full dataset

First we import the necessary libraries

In [ ]:
import csv
import pandas as pd
import random
import pyarrow.feather as feather
import regex as re
from cleantext import clean
import pyarrow.feather as feather
from multiprocessing import Pool
import gc
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import time

We load the full data set

In [ ]:
full_data = pd.read_csv("data/news_cleaned_2018_02_13-1.csv", on_bad_lines='skip', index_col=0)

In [ ]:
len(full_data)

We get a sample consisting of 10% of the full dataset

In [ ]:
full_data_sample.head()

In [ ]:
len(full_data_sample)

Next, we make sure to delete the full data set from memory such that we can save our sample

In [ ]:
import gc

In [ ]:
del full_data
gc.collect()

Then, we save the sample to csv format

In [ ]:
full_data_sample.to_csv("data/sample.csv")

### Inspecting The Sample

In [ ]:
typedict = {}
for i in data['type']: 
    if str(i) in typedict:
        typedict[str(i)] +=1 
    else: 
        typedict[str(i)] =1 
typedict
typedictperc = typedict.copy()
#laver et nyt dictionairy som viser det i procenttal
for i in typedictperc:
    typedictperc[i] = (typedictperc[i]/(len(data)))*100

typedictperc

In [ ]:
allArticlesCount = 928083 + 146080 + 1300444 + 905981 + 144939 + 117374 + 292201 + 2435471 + 319830 + 1920139
#Jeg henter data from README.md om hvor mange artikler af hver type, der er i det fulde datasæt
realtypeperc = {"reliable": (1920139/allArticlesCount)*100, "political": (2435471/allArticlesCount)*100, "bias": (1300444/allArticlesCount)*100, "Satire": (146080/allArticlesCount)*100, "fake": (928083/allArticlesCount)*100, "conspiracy":(905981/allArticlesCount)*100, "unreliable": (319830/allArticlesCount)*100, "clickbait":(292201/allArticlesCount)*100, "junksci":(144939/allArticlesCount)*100 , "hate":(117374/allArticlesCount)*100}
realtypeperc
#jeg præsenterer det som procenter
#ved sammenligning er det tydeligt, at Mikkels forkortede datasæt ikke har markant anderledes proportion ift. artikeltyper, end det fulde datasæt
#Største forskel er at der er tilføjet nye typer af artikler som rumor til datasættet, siden README blev skrevet. 
#Vi kan bruge dette til at forsikre os selv om at vores forkortede datasæt er repræsentativt for det fulde datasæt. 

### Structure Sample For Preprocessing

Before we preprocess there are some articles we can remove from the data set. We also change the labels so that
they are either 'reliable' or 'fake'

In [ ]:
def structure_dataframe(df): 
    # set all lables to either 'reliable' or 'fake' using the rules below
    df.type = df.type.replace({'political': 'reliable', 'junksci': 'fake', 'bias' : 'fake', 'satire': 'fake', 'conspiracy': 'fake', 'rumor': 'fake', 'unreliable' : 'fake', 'clickbait': 'fake', 'hate': 'fake'})
    # there are some junk labels due to bad formatting, so we remove those
    df = df[(df.type == 'reliable') |(df.type == 'fake')]
    # remove all NaN types
    df = df[df.type.notnull()]
    # remove all NaN content
    df = df[df.content.notnull()]
    # remove duplicates. Some entries have simmilar content, like a notification from Google Plus
    # or a message saying that the TOR browser needs to be installed in order view the article.
    df = df.drop_duplicates(subset = 'content', keep = 'last')
    
    df = df.reset_index()
    return df

In [ ]:
full_data_sample = structure_dataframe(full_data_sample)

In [ ]:
def labelperc(df): 
    labeldict = {}
    for i in df.type: 
        if i in labeldict: 
            labeldict[i] +=1
        else: 
            labeldict[i] = 1
    for i in labeldict: 
        labeldict[i] = labeldict[i]/len(df)*100
    return labeldict

In [ ]:
labelperc(full_data_sample)

In [ ]:
full_data_sample.to_csv("data/sample_STRUCTURED.csv")

### Preprocessing the larger sample

Then, we preprocess the sample

In [ ]:
file_name = "sample_preprocessed_ver_17_03_22"
for chunck in pd.read_csv("data/sample_STRUCTURED.csv", chunksize=10000):
    clean_dataframe(chunck)
    preprocess(chunck)
    start = time.time()
    chunck.to_csv(file_name, mode='a')
    end = time.time()
    print("writing to csv took " + str(end - start) + " seconds")